In [1]:
import json
import os
import sys
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_confusion_matrix, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", '-')
words_by_grade[3][:10]

[('able', 'A-B-L-E'),
 ('above', 'A-B-O-V-E'),
 ('afraid', 'A-F-R-A-I-D'),
 ('afternoon', 'A-F-T-E-R-N-O-O-N'),
 ('again', 'A-G-A-I-N'),
 ('age', 'A-G-E'),
 ('air', 'A-I-R'),
 ('airplane', 'A-I-R-P-L-A-N-E'),
 ('almost', 'A-L-M-O-S-T'),
 ('alone', 'A-L-O-N-E')]

In [2]:
with open("../data/grade_spelling_eval_results_gptj.json") as file:
    eval_results = json.load(file)

In [3]:
table = create_table(eval_results[FULL_SPELLING_EVAL.name]['2']['data'])
table

,Expected 1,Result 1,Expected 2,Result 2,Expected 3,Result 3,Expected 4,Result 4,Expected 5,Result 5
0,A-L-L,A-L-L,A-M,A-M,A-N-D,A-N-D,A-T,A-T,B-A-L-L,B-A-L-L
1,B-E,B-E,B-E-D,B-E-D,B-I-G,B-I-G,B-O-O-K,B-O-O-K,B-O-X,B-O-X
2,B-O-Y,B-O-Y,B-U-T,B-U-T,C-A-M-E,C-A-M,C-A-N,C-A-N,C-A-R,C-A-R
3,C-A-T,C-A-T,C-O-M-E,C-O-M-E,C-O-W,C-O-W,D-A-D,D-A-D,D-A-Y,D-A-Y
4,D-I-D,D-I-D,D-O,D-O,D-O-G,D-O-G,F-A-T,F-A-T,F-O-R,F-O
5,F-U-N,F-U-N,G-E-T,G-E-T,G-O,G-O,G-O-O-D,G-O-O-D,G-O-T,G-O-T
6,H-A-D,H-A-D,H-A-T,H-A-T,H-E,H-E,H-E-N,H-E-N,H-E-R-E,H-E-R-E
7,H-I-M,H-I-M,H-I-S,H-I-S,H-O-M-E,H-O-M-E,H-O-T,H-O-T,I-F,I-F
8,I-N,I-N,I-N-T-O,I-N-T-O,I-S,I-S,I-T,I-T,I-T-S,I-T-S
9,L-E-T,L-E-T,L-I-K-E,L-I-K-E,L-O-O-K,L-O-K-E,M-A-N,M-A-N,M-A-Y,M-A-Y


In [4]:
from collections import defaultdict

char_dict = {item: {'correct': 0, 'incorrect': 0} for item in string.ascii_uppercase}

def populate_char_dict(char_dict, answer, response):
    answer, response = answer.split('-'), response.split('-')
    for i, char in enumerate(answer):
        if char in char_dict:
            if len(response) > i and response[i] == answer[i]:
                char_dict[char]['correct'] += 1
            else:
                char_dict[char]['incorrect'] += 1
                break

# Check for the first misspelling in all words without special characters like periods/apostrophes.
# Only the first because otherwise misspellings like R-A-B-B-I-T -> R-A-B-I-T would count as wrong for I and T as well.
for grade in eval_results[FULL_SPELLING_EVAL.name]['2']['data']:
    for word in eval_results[FULL_SPELLING_EVAL.name]['2']['data'][grade]:
        if ''.join(word['answer'].split('-')).isalpha():
            populate_char_dict(char_dict, word['answer'], word['formatted_response'])

print(char_dict)

{'A': {'correct': 265, 'incorrect': 10}, 'B': {'correct': 80, 'incorrect': 2}, 'C': {'correct': 107, 'incorrect': 3}, 'D': {'correct': 125, 'incorrect': 2}, 'E': {'correct': 372, 'incorrect': 41}, 'F': {'correct': 69, 'incorrect': 3}, 'G': {'correct': 97, 'incorrect': 8}, 'H': {'correct': 135, 'incorrect': 5}, 'I': {'correct': 212, 'incorrect': 13}, 'J': {'correct': 4, 'incorrect': 0}, 'K': {'correct': 50, 'incorrect': 3}, 'L': {'correct': 161, 'incorrect': 26}, 'M': {'correct': 89, 'incorrect': 2}, 'N': {'correct': 214, 'incorrect': 15}, 'O': {'correct': 246, 'incorrect': 21}, 'P': {'correct': 99, 'incorrect': 2}, 'Q': {'correct': 1, 'incorrect': 0}, 'R': {'correct': 213, 'incorrect': 13}, 'S': {'correct': 185, 'incorrect': 7}, 'T': {'correct': 239, 'incorrect': 16}, 'U': {'correct': 95, 'incorrect': 7}, 'V': {'correct': 39, 'incorrect': 1}, 'W': {'correct': 86, 'incorrect': 3}, 'X': {'correct': 9, 'incorrect': 0}, 'Y': {'correct': 69, 'incorrect': 1}, 'Z': {'correct': 2, 'incorrect':

In [5]:
import plotly.io as pio

pio.renderers.default = "iframe" # No idea why this works, and the others don't.
char_accuracy_dict = {i: char_dict[i]['correct'] / (char_dict[i]['correct'] + char_dict[i]['incorrect']) for i in char_dict.keys()}
print(char_accuracy_dict)
basic_bar_graph(char_accuracy_dict)

{'A': 0.9636363636363636, 'B': 0.975609756097561, 'C': 0.9727272727272728, 'D': 0.984251968503937, 'E': 0.9007263922518159, 'F': 0.9583333333333334, 'G': 0.9238095238095239, 'H': 0.9642857142857143, 'I': 0.9422222222222222, 'J': 1.0, 'K': 0.9433962264150944, 'L': 0.8609625668449198, 'M': 0.978021978021978, 'N': 0.9344978165938864, 'O': 0.9213483146067416, 'P': 0.9801980198019802, 'Q': 1.0, 'R': 0.9424778761061947, 'S': 0.9635416666666666, 'T': 0.9372549019607843, 'U': 0.9313725490196079, 'V': 0.975, 'W': 0.9662921348314607, 'X': 1.0, 'Y': 0.9857142857142858, 'Z': 1.0}


In [6]:
from collections import defaultdict
import copy

allowed_characters = list(string.ascii_uppercase) + ['_']
char_dict = {item: defaultdict(int) for item in allowed_characters}
char_dict['Other'] = defaultdict(int)

def populate_char_dict(char_dict, answer, response):
    # answer and response are of the form H-E-L-L-O.
    answer, response = answer.split('-'), response.split('-')

    for i, char in enumerate(answer):
        # If response is shorter than answer, treat missing characters as spaces
        if char not in char_dict:
            char = 'Other'
        resp_char = response[i] if len(response) > i else '_'
        if resp_char not in char_dict:
            resp_char = 'Other'
        char_dict[char][resp_char] += 1

        if char != resp_char:
            break  # Stop checking further characters after the first misspelling

# Check for the first misspelling in all words without special characters like periods/apostrophes.
# Only the first because otherwise misspellings like R-A-B-B-I-T -> R-A-B-I-T would count as wrong for I and T as well.
for grade in eval_results[FULL_SPELLING_EVAL.name]['2']['data']:
    for word in eval_results[FULL_SPELLING_EVAL.name]['2']['data'][grade]:
        if ''.join(word['answer'].split('-')).isalpha(): # Ignore non-alphabetic characters like apostrophes.
            populate_char_dict(char_dict, word['answer'], word['formatted_response'])
            
create_confusion_matrix(char_dict)

In [10]:
def get_precision(char_dict, char):
    true_positives = char_dict[char][char]
    false_positives = sum([char_dict[c][char] for c in char_dict if c!= char])
    return 0 if true_positives + false_positives == 0 else round(true_positives / (true_positives + false_positives), 3)

def get_recall(char_dict, char):
    true_positives = char_dict[char][char]
    false_negatives = sum([char_dict[char][c] for c in char_dict if c != char])
    return 0 if true_positives + false_negatives == 0 else round(true_positives / (true_positives + false_negatives), 3)

def get_f1_score(char_dict, char):
    precision, recall = get_precision(char_dict, char), get_recall(char_dict, char)
    return 0 if precision + recall == 0 else round((2 * precision * recall) / (precision + recall), 3)

prf_dict = {char: {'precision': get_precision(char_dict, char), 
                   'recall': get_recall(char_dict, char), 
                   'f1_score': get_f1_score(char_dict, char)} for char in char_dict}

basic_bar_graph({k: prf_dict[k]['precision'] for k in prf_dict.keys()}, title="Precision")

In [11]:
basic_bar_graph({k: prf_dict[k]['recall'] for k in prf_dict.keys()}, title="Recall")

In [12]:
basic_bar_graph({k: prf_dict[k]['f1_score'] for k in prf_dict.keys()}, title="F1 Score")

In [19]:
import plotly.graph_objs as go

# Assuming 'f1_scores', 'precisions', and 'recalls' are lists containing the F1 score,
# precision, and recall for each category respectively. 'categories' is a list of category names.

# Create a bar plot for F1 scores
bar_f1 = go.Bar(
    x=list(prf_dict.keys()),
    y=[prf_dict[k]['f1_score'] for k in prf_dict.keys()],
    name='F1 Score',
    # Other styling options
)

# Create a scatter plot for precision
scatter_precision = go.Scatter(
    x=list(prf_dict.keys()),
    y=[prf_dict[k]['precision'] for k in prf_dict.keys()],
    mode='markers',
    name='Precision',
    line=dict(color='rgba(255, 0, 0, .9)', width=6),
    # Other styling options
)

# Create a scatter plot for recall
scatter_recall = go.Scatter(
    x=list(prf_dict.keys()),
    y=[prf_dict[k]['recall'] for k in prf_dict.keys()],
    mode='markers',
    name='Recall',
    line=dict(color='rgba(0, 192, 0, .9)', width=6),
    # Other styling options
)

# Create a layout
layout = go.Layout(
    title='F1, Precision, and Recall',
    xaxis=dict(title='Category'),
    yaxis=dict(title='Scores'),
    # Other layout options
)

# Combine the plots
fig = go.Figure(data=[bar_f1, scatter_precision, scatter_recall], layout=layout)

# Show the figure
fig.show()


In [29]:
def get_results_by_word_length(eval_result, should_print=False):
    length_results = {i: {'correct': 0, 'incorrect': 0} for i in range(1, 13)}
    for grade in eval_result:
        for word in eval_result[grade]:
            length = len(word['answer'].replace('-', ''))
            correct = word['answer'] in word['formatted_response']
            if correct:
                length_results[length]['correct'] += 1
            else:
                length_results[length]['incorrect'] += 1
    length_accuracies = {i: 0 if (length_results[i]['correct'] + length_results[i]['incorrect']) == 0 else length_results[i]['correct'] / (length_results[i]['correct'] + length_results[i]['incorrect']) for i in length_results.keys()}
    if should_print:
        print(length_results)
    basic_bar_graph(length_accuracies, title="Accuracy By Word Length")

get_results_by_word_length(eval_results[FULL_SPELLING_EVAL.name]['2']['data'])

{1: {'correct': 0, 'incorrect': 0}, 2: {'correct': 25, 'incorrect': 0}, 3: {'correct': 118, 'incorrect': 8}, 4: {'correct': 222, 'incorrect': 50}, 5: {'correct': 133, 'incorrect': 62}, 6: {'correct': 74, 'incorrect': 46}, 7: {'correct': 33, 'incorrect': 32}, 8: {'correct': 9, 'incorrect': 19}, 9: {'correct': 3, 'incorrect': 11}, 10: {'correct': 1, 'incorrect': 0}, 11: {'correct': 0, 'incorrect': 1}, 12: {'correct': 0, 'incorrect': 0}}


In [38]:
def is_double_letter(word):
    return True in [word['word'][i] == word['word'][i+1] for i in range(len(word['word'])-1)]

def get_results_by_double_letter(eval_result, should_print=False):
    doubles = {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 0, 'incorrect': 0}}
    for grade in eval_result:
        for word in eval_result[grade]:
            correct = word['answer'] in word['formatted_response']
            if correct:
                doubles[is_double_letter(word)]['correct'] += 1
            else:
                doubles[is_double_letter(word)]['incorrect'] += 1
    double_accuracies = {i: 0 if (doubles[i]['correct'] + doubles[i]['incorrect']) == 0 else doubles[i]['correct'] / (doubles[i]['correct'] + doubles[i]['incorrect']) for i in doubles.keys()}
    if should_print:
        print(doubles)
    basic_bar_graph(double_accuracies, title="Accuracy By Double Letter")

get_results_by_double_letter(eval_results[FULL_SPELLING_EVAL.name]['2']['data'], should_print=True)

{True: {'correct': 78, 'incorrect': 48}, False: {'correct': 540, 'incorrect': 181}}


{True: {'correct': 78, 'incorrect': 48},
 False: {'correct': 540, 'incorrect': 181}}

In [46]:
def get_results_by_length_and_double_letter(eval_result, should_print=False):
    doubles = {i: {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 0, 'incorrect': 0}} for i in range(1, 12)}
    for grade in eval_result:
        for word in eval_result[grade]:
            correct = word['answer'] in word['formatted_response']
            length = length = len(word['answer'].replace('-', ''))
            if correct:
                doubles[length][is_double_letter(word)]['correct'] += 1
            else:
                doubles[length][is_double_letter(word)]['incorrect'] += 1
    double_accuracies = {}
    for i in doubles.keys():
        double_accuracies[f"{i}, {True}"] = 0
        double_accuracies[f"{i}, {False}"] = 0
    for k in doubles.keys():
        for i in doubles[k].keys():
            total = doubles[k][i]['correct'] + doubles[k][i]['incorrect']
            double_accuracies[f"{k}, {i}"] = 0 if total == 0 else doubles[k][i]['correct'] / total
    if should_print:
        print(doubles)
        print(double_accuracies)
    basic_bar_graph(double_accuracies, title="Accuracy By Double Letter")

get_results_by_length_and_double_letter(eval_results[FULL_SPELLING_EVAL.name]['2']['data'], should_print=True)

{1: {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 0, 'incorrect': 0}}, 2: {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 25, 'incorrect': 0}}, 3: {True: {'correct': 7, 'incorrect': 1}, False: {'correct': 111, 'incorrect': 7}}, 4: {True: {'correct': 28, 'incorrect': 15}, False: {'correct': 194, 'incorrect': 35}}, 5: {True: {'correct': 20, 'incorrect': 6}, False: {'correct': 113, 'incorrect': 56}}, 6: {True: {'correct': 14, 'incorrect': 10}, False: {'correct': 60, 'incorrect': 36}}, 7: {True: {'correct': 6, 'incorrect': 8}, False: {'correct': 27, 'incorrect': 24}}, 8: {True: {'correct': 3, 'incorrect': 3}, False: {'correct': 6, 'incorrect': 16}}, 9: {True: {'correct': 0, 'incorrect': 5}, False: {'correct': 3, 'incorrect': 6}}, 10: {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 1, 'incorrect': 0}}, 11: {True: {'correct': 0, 'incorrect': 0}, False: {'correct': 0, 'incorrect': 1}}}
{'1, True': 0, '1, False': 0, '2, True': 0, '2, False': 1.0, '3, True': 0.87